# Implement compute session CSD

In [ ]:
import json
import time
import pandas as pd
import src.paths
from src.data import load_ephys
from src.lfp import remove_dc_offset, combine_neighbors, decimate_by_2
from src.lfp import bandpass_1_300Hz, notch_60Hz, compute_csd


def raw_lfp_to_csd(X):
    X = remove_dc_offset(X)
    X = combine_neighbors(X)
    X = bandpass_1_300Hz(X)
    X = notch_60Hz(X)
    X = decimate_by_2(X)
    X = compute_csd(X)
    X = combine_neighbors(X)
    return X


def compute_session_csd(date, region):
    with open(src.paths.REF / 'date_recoffset.json', 'r') as f:
        offsets = json.load(f)
    
    suffix = {'hpc': '.imec1.lf.bin', 'pfc': '.imec0.lf.bin'}
    file = str(src.paths.session_path(date)) + suffix
    path = src.paths.DATA / date / file
    lfpstart, lfpend = offsets[date]
    duration = (lfpend - lfpstart) / 2500 / 60
    
    print(f'date: {date}')
    print(f'reading path: {path}')
    print(f'lfpstart, lfpend:', lfpstart, lfpend)
    print(f'duration: {duration} min')

    intervals = []
    startmin = 0
    while True:
        endmin = min(startmin + 1, duration)
        intervals.append((startmin, endmin))
        if endmin == duration:
            break
        startmin = endmin
    print(f'# batches:', len(intervals))

    t0 = time.time()
    l = []
    for i, (start, end) in enumerate(intervals):
        raw = load_ephys(path, startmin=start, endmin=end, offset=lfpstart, cleaned=True)
        X = raw_lfp_to_csd(raw.values)
        idx = raw.index[::2]
        col = raw.columns[::4] // 4
        csd = pd.DataFrame(X, index=idx, columns=col)
        l.append(csd)
        if i % 10 == 0:
            print(f'finished batch {i} ({time.time() - t0:0.2f} s)')
    print('done')

    csd = pd.concat(l, axis=0)
    print('total samples:', len(csd))
    print('GB:', csd.values.nbytes / 1e9)
    return csd

    csd.to_pickle(outfile)